In [2]:
import pandas as pd
raw=pd.read_csv('Download_data_RC.txt',sep='\t',dtype=str)
unique_raw_ids = raw['Raw_ID2'].dropna().unique()
print(len(unique_raw_ids))
compound_id=[]
for raw in unique_raw_ids:
    compound_id.append([raw.split(':')[0],raw.split(':')[1],''])

363


In [13]:
import requests



def search_pubchem(cid):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/IsomericSMILES/JSON"
    response = requests.get(url)
    if response.status_code == 200:
        smiles = response.json()["PropertyTable"]["Properties"][0]["IsomericSMILES"]
        return smiles
    else:
        return -1
    


In [20]:
import requests

drugbank_id = "DB00020"
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/sourceid/DRUGBANK/{drugbank_id}/cids/JSON"

response = requests.get(url)
if response.status_code == 200:
    cid = response.json()["IdentifierList"]["CID"][0]  # 获取 PubChem CID
    smiles_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/IsomericSMILES/JSON"
    smiles_response = requests.get(smiles_url)
    if smiles_response.status_code == 200:
        smiles = smiles_response.json()["PropertyTable"]["Properties"][0]["IsomericSMILES"]
        print(f"DrugBank ID {drugbank_id} 的 SMILES: {smiles}")
else:
    print("未找到 CID")

未找到 CID


In [17]:
import tqdm
import time
success=0
fail=0
for row in tqdm.tqdm(compound_id):   
    response=-1    
    time.sleep(1)
   
    if row[0]=='CID':
        response=search_pubchem(row[1])
        if response!=-1:
            row[2]=response
            success+=1
        else:
            row[2]='no result'
            fail+=1
   
print(f'\n success : {success}, fail : {fail}')

100%|██████████| 363/363 [08:02<00:00,  1.33s/it]


 success : 351, fail : 0


In [22]:
compound_df=pd.DataFrame(compound_id,columns=['database','id','SMILES'])
compound_df.to_csv('compound.csv')

In [1]:
import pandas as pd
compound_df=pd.read_csv('compound.csv')
compound=[]
for _,row in compound_df.iterrows():
    if row['database']=='CID' :
        compound.append([row['id'],row['SMILES'],None])

In [ ]:
compound

In [2]:
import os
from dataset.smiles2graph import smiles2graph
import json

os.makedirs('img', exist_ok=True)
idx=0
for row in compound:
    row[2]=smiles2graph(str(row[1]))
    idx+=1
graph=[json.dumps(row[2],default=lambda x: x.tolist())for row in compound]

In [32]:
graph

['{"edge_index": [[2, 3, 3, 4], [3, 2, 4, 3]], "edge_feat": [[0, 0], [0, 0], [0, 0], [0, 0]], "node_feat": [[6, 0], [6, 0], [16, 0], [77, 0], [16, 0]], "num_nodes": 5}',
 '{"edge_index": [[0, 1, 1, 2, 2, 3, 3, 4, 3, 5, 5, 6, 6, 7, 2, 8, 8, 9, 9, 10, 10, 11, 11, 12, 11, 13, 13, 14, 10, 15, 9, 16, 9, 17, 6, 0, 12, 8], [1, 0, 2, 1, 3, 2, 4, 3, 5, 3, 6, 5, 7, 6, 8, 2, 9, 8, 10, 9, 11, 10, 12, 11, 13, 11, 14, 13, 15, 10, 16, 9, 17, 9, 0, 6, 8, 12]], "edge_feat": [[3, 0], [3, 0], [3, 0], [3, 0], [3, 0], [3, 0], [1, 0], [1, 0], [3, 0], [3, 0], [3, 0], [3, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [3, 0], [3, 0], [0, 0], [0, 0]], "node_feat": [[5, 0], [5, 0], [6, 0], [5, 0], [7, 0], [6, 0], [5, 0], [6, 0], [5, 1], [5, 0], [5, 1], [5, 2], [7, 0], [5, 0], [7, 0], [7, 0], [8, 0], [8, 0]], "num_nodes": 18}',
 '{"edge_index": [[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6,

In [8]:

from pipeline.models import gnn
model_path = "ckpt/gcn_contextpred.pth"
model = gnn.GNN(num_layer=5, emb_dim=300, gnn_type='gcn', use_graph_agg=True)#remove pipeline.tasks.eval for environment
model.load_from_pretrained(url_or_filename=model_path)
model=model.eval()


OSError: dlopen(/Users/gaoyuanji/miniconda3/envs/drugchat/lib/python3.9/site-packages/torch_scatter/_version_cpu.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <00AB9889-17D0-3360-8013-21636ADEFE10> /Users/gaoyuanji/miniconda3/envs/drugchat/lib/python3.9/site-packages/torch_scatter/_version_cpu.so
  Expected in:     <D983C0C6-7C01-3D9C-A96B-07BB20D2CE66> /Users/gaoyuanji/miniconda3/envs/drugchat/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib

In [37]:
model(graph[0])

AttributeError: 'str' object has no attribute 'x'